En este notebook encontrarán los pasos e instrucciones necesarias para para realizar el tratamiento de los datos.

In [49]:
# Importar librerías de python
import pandas as pd
import numpy as np

### Extracting data

In [50]:
def check_dataframe(*dfs):
    """
    Verifica si los DataFrames tienen la misma cantidad y nombre de columnas, y muestra la cantidad y nombres de las columnas.

    Args:
        *dfs: uno o más DataFrames

    Returns:
        bool: True si tienen las mismas columnas y cantidad, False si no.
    """
    cols = None
    for i, df in enumerate(dfs):
        if cols is None:
            cols = set(df.columns)
            num_cols = len(cols)
            print(f"DataFrame {i+1} tiene {num_cols} columnas: {', '.join(cols)}")
        elif cols != set(df.columns):
            print(f"DataFrame {i+1} tiene {len(df.columns)} columnas diferentes: {', '.join(set(df.columns) - cols)}")
            return False
    return True

In [51]:
# crear los DataFrames (df_a, df_d, df_h, df_n)
df_a = pd.read_csv("raw/amazon_prime_titles.csv")
df_d = pd.read_csv("raw/disney_plus_titles.csv")
df_h = pd.read_csv("raw/hulu_titles.csv")
df_n = pd.read_csv("raw/netflix_titles.csv")

if check_dataframe(df_a, df_d, df_h, df_n):
    print("Los 4 DataFrames tienen las mismas columnas y cantidad")
else:
    print("Los 4 DataFrames no tienen las mismas columnas y/o cantidad")

DataFrame 1 tiene 12 columnas: listed_in, release_year, cast, title, country, date_added, duration, description, show_id, type, rating, director
Los 4 DataFrames tienen las mismas columnas y cantidad


In [52]:
# Observamos uno de los dataframe 
df_a.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...


### Transformation data

#### Plataformas

1. `Generar campo id:` Cada id se compondrá de la primera letra del nombre de la plataforma, seguido del show_id ya presente en los datasets (ejemplo para títulos de Amazon = as123).

In [53]:
#Crear la columna con la generación del "id"
df_a["id"] = "a" + df_a["show_id"].astype(str)
df_d["id"] = "d" + df_d["show_id"].astype(str)
df_h["id"] = "h" + df_h["show_id"].astype(str)
df_n["id"] = "n" + df_n["show_id"].astype(str)

In [54]:
# Generar una la columna platform
df_a["platform"] = "amazon" 
df_d["platform"] = "disney" 
df_h["platform"] = "hulu" 
df_n["platform"] = "netflix" 

In [55]:
# Concatenar todos los dataframe con la función "concat"
df_plataformas = pd.concat([df_a, df_d, df_h, df_n], ignore_index= True)

In [56]:
df_plataformas.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,id,platform
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...,as1,amazon


2.	Los `valores nulos` del campo rating deberán reemplazarse por el string “G” (corresponde al maturity rating: “general for all audiences”.)

In [57]:
# reemplazar los valores nulos en la columna "rating" con un nuevo valor "G" 
df_plataformas["rating"] = df_plataformas["rating"].fillna('G')

In [58]:
# Observamos los valores únicos de la columna rating
df_plataformas.rating.unique()

array(['G', '13+', 'ALL', '18+', 'R', 'TV-Y', 'TV-Y7', 'NR', '16+',
       'TV-PG', '7+', 'TV-14', 'TV-NR', 'TV-G', 'PG-13', 'TV-MA', 'PG',
       'NC-17', 'UNRATED', '16', 'AGES_16_', 'AGES_18_', 'ALL_AGES',
       'NOT_RATE', 'TV-Y7-FV', 'NOT RATED', '2 Seasons', '93 min',
       '4 Seasons', '136 min', '91 min', '85 min', '98 min', '89 min',
       '94 min', '86 min', '3 Seasons', '121 min', '88 min', '101 min',
       '1 Season', '83 min', '100 min', '95 min', '92 min', '96 min',
       '109 min', '99 min', '75 min', '87 min', '67 min', '104 min',
       '107 min', '84 min', '103 min', '105 min', '119 min', '114 min',
       '82 min', '90 min', '130 min', '110 min', '80 min', '6 Seasons',
       '97 min', '111 min', '81 min', '49 min', '45 min', '41 min',
       '73 min', '40 min', '36 min', '39 min', '34 min', '47 min',
       '65 min', '37 min', '78 min', '102 min', '129 min', '115 min',
       '112 min', '61 min', '106 min', '76 min', '77 min', '79 min',
       '157 min', '28 mi

In [60]:
def mover_valores(df, col_origen, col_destino, sufijos):
    # Usamos la condición para identificar los valores que deben moverse
    mask = df[col_origen].apply(lambda x: x.endswith(sufijos))
    
    # Movemos los valores a la columna destino
    df.loc[mask, col_destino] = df.loc[mask, col_origen]
    
    # Eliminamos los valores movidos de la columna origen y lo reemplazamos por "G"
    df.loc[mask, col_origen] = 'G'
    
    return df

In [63]:
#mover_valores(df_plataformas, 'rating', 'duration', ('Season','Seasons', 'min'))

In [64]:
# Observamos los valores unicos de la columna rating
df_plataformas.rating.unique()

array(['G', '13+', 'ALL', '18+', 'R', 'TV-Y', 'TV-Y7', 'NR', '16+',
       'TV-PG', '7+', 'TV-14', 'TV-NR', 'TV-G', 'PG-13', 'TV-MA', 'PG',
       'NC-17', 'UNRATED', '16', 'AGES_16_', 'AGES_18_', 'ALL_AGES',
       'NOT_RATE', 'TV-Y7-FV', 'NOT RATED', 'UR'], dtype=object)

3. De haber fechas, deberán tener el `formato AAAA-mm-dd`

In [65]:
# convertir la columna "date_added" a tipo datetime
df_plataformas["date_added"] = pd.to_datetime(df_plataformas["date_added"])

4.	Los `campos de texto` deberán estar en `minúsculas`, sin excepciones

In [66]:
df_plataformas = df_plataformas.apply(lambda x: x.str.lower() if x.dtype == "object" else x)

5.	El campo `duration` debe `convertirse en dos campos: duration_int y duration_type`. El primero será un integer y el segundo un string indicando la unidad de medición de duración: min (minutos) o season (temporadas)

In [67]:
df_plataformas.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,id,platform
0,s1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113 min,"comedy, drama",a small fishing village must procure a local d...,as1,amazon
1,s2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,110 min,"drama, international",a metro family decides to fight a cyber crimin...,as2,amazon


In [68]:
# Con el método "str.split()" podemos separar la columna "duration" en dos columnas "duration_int" y "duration_type"
df_plataformas[["duration_int","duration_type"]] = df_plataformas["duration"].str.split(" ", expand = True)

# Cambiamos el tipo de dato de 'duration_int' a int
df_plataformas["duration_int"] = df_plataformas["duration_int"].astype("Int64")

In [69]:
df_plataformas["duration_type"].unique()

array(['min', 'season', 'seasons', nan], dtype=object)

In [70]:
df_plataformas['duration_type'] = df_plataformas['duration_type'].replace({'seasons': 'season'})

In [71]:
# Verificar si hay valores duplicados
df_plataformas.duplicated().sum()

0

In [72]:
df_plataformas.isnull().sum()

show_id              0
type                 0
title                0
director          8259
cast              5321
country          11499
date_added        9554
release_year         0
rating               0
duration           187
listed_in            0
description          4
id                   0
platform             0
duration_int       187
duration_type      187
dtype: int64

In [73]:
df_plataformas.dtypes

show_id                  object
type                     object
title                    object
director                 object
cast                     object
country                  object
date_added       datetime64[ns]
release_year              int64
rating                   object
duration                 object
listed_in                object
description              object
id                       object
platform                 object
duration_int              Int64
duration_type            object
dtype: object

In [74]:
df_plataformas.shape

(22998, 16)

#### Rating

In [26]:
# crear los DataFrames (df_r1, df_r2, df_r3, df_r4,df_r5, df_r6, df_r7, df_r8)
df_r1 = pd.read_csv("raw/ratings/1.csv")
df_r2 = pd.read_csv("raw/ratings/2.csv")
df_r3 = pd.read_csv("raw/ratings/3.csv")
df_r4 = pd.read_csv("raw/ratings/4.csv")
df_r5 = pd.read_csv("raw/ratings/5.csv")
df_r6 = pd.read_csv("raw/ratings/6.csv")
df_r7 = pd.read_csv("raw/ratings/7.csv")
df_r8 = pd.read_csv("raw/ratings/8.csv")

if check_dataframe(df_r1, df_r2, df_r3, df_r4,df_r5, df_r6, df_r7, df_r8):
    print("Los 4 DataFrames tienen las mismas columnas y cantidad")
else:
    print("Los 4 DataFrames no tienen las mismas columnas y/o cantidad")

DataFrame 1 tiene 4 columnas: movieId, userId, timestamp, rating
Los 4 DataFrames tienen las mismas columnas y cantidad


In [27]:
# Concatenando todos los dataframe con la función "concat"
df_rating = pd.concat([df_r1, df_r2, df_r3,df_r4, df_r5, df_r6, df_r7, df_r8], ignore_index= True)

In [28]:
df_rating.shape

(11024289, 4)

In [29]:
df_rating.head(2)

,userId,rating,timestamp,movieId
0,1,1.0,1425941529,as680
1,1,4.5,1425942435,ns2186


De haber fechas, deberán tener el `formato AAAA-mm-dd`

In [30]:
# Convertir la columna 'fechas' al formato AAAA-mm-dd
df_rating["timestamp"] = pd.to_datetime(df_rating["timestamp"], unit="s").dt.strftime('%Y-%m-%d')

# convertir la columna "date_added" a tipo datetime
df_rating["timestamp"] = pd.to_datetime(df_rating["timestamp"])

In [31]:
# Renombrar los nombres de las columnas 
df_rating = df_rating.rename(columns={"rating": "score", "movieId": "id"})

In [32]:
df_rating.head(2)

,userId,score,timestamp,id
0,1,1.0,2015-03-09,as680
1,1,4.5,2015-03-09,ns2186


In [33]:
df_rating.dtypes

userId                int64
score               float64
timestamp    datetime64[ns]
id                   object
dtype: object

In [34]:
# Verificar si hay valores duplicados
df_rating.duplicated().sum()

10466

In [35]:
df_rating = df_rating.drop_duplicates()

In [36]:
df_rating.isnull().sum()

userId       0
score        0
timestamp    0
id           0
dtype: int64

In [37]:
df_rating.shape

(11013823, 4)

### Promedio de score

In [38]:
# Obtenemos el promedio de rating utilizando el método groupby
average_score = df_rating.groupby("id")["score"].mean().round(2)

# Convertirmos a dataframe
average_score = pd.DataFrame({"id":average_score.index, "average_score": average_score.values})

In [39]:
average_score.head(2)

,id,average_score
0,as1,3.47
1,as10,3.44


In [40]:
average_score.shape

(22998, 2)

### Datasets para la API y modelo ML

In [75]:
# Unimos los dataframes utilizando el método merge. El mismo que se empleará para la API.
df_platforms = pd.merge(df_plataformas,average_score, on = "id")
df_platforms.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,id,platform,duration_int,duration_type,average_score
0,s1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113 min,"comedy, drama",a small fishing village must procure a local d...,as1,amazon,113,min,3.47


In [77]:
# Eliminar columnas que no se necesita para la API
df_platforms = df_platforms.drop(['show_id','date_added','rating','duration'], axis=1)

In [78]:
# Exportamos el dataframe df_platforms en formato csv
df_platforms.to_csv('df_platforms.csv', index=False)

In [89]:
# Columnas que deseas mantener de df_plataformas
col_plataformas = ["id", "title", "type", "platform"]

# Unimos los dataframes utilizando el método merge. El mismo que se empleará para el modelo ML.
df_score = pd.merge(df_rating, df_plataformas[col_plataformas], on="id")

In [91]:
# Exportamos el dataframe df_plataformas en formato csv
df_score.to_csv('df_score.csv', index=False)